# Tensorflow Hello World

In [19]:
import tensorflow as tf

x= tf.constant([100,200,300], name='x')
y=tf.constant([1,2,3],name='y')
sum_x=tf.reduce_sum(x,name='sum_x')
prod_y=tf.reduce_prod(y,name='prod_y')
final_div=tf.div(sum_x,prod_y,name='final_div')

sess=tf.Session()
print ("sum(x) :{}".format( sess.run(sum_x) ))
print ("prod(y): {}".format( sess.run(prod_y)))
print ("final_div: {}".format( sess.run(final_div)))

writer = tf.summary.FileWriter('./tbout/example',sess.graph)
sess.close()

sum(x) :600
prod(y): 6
final_div: 100


# Constant, placeholder and session

Setup non-changing values by using tf.constant(self explanatory) and tf.placeholder( a place holder whose values will be passed in via feed_dict in session.run. Values passed in for tf.placeholder cannot be change during the session.

A debugging and visualization tool tensor board can be used by first writing the session graph into a folder like this :

`writer = tf.summary.FileWriter('./another',sess.graph)`

and then running tensorboard : 
 `tensorboard --logdir=another`

In [17]:
import tensorflow as tf

W=tf.constant([10,100], name='const_W')
x=tf.placeholder(tf.int32, name='x')
b=tf.placeholder(tf.int32,name='b')

Wx=tf.multiply(W,x,name='Wx')
y=tf.add(Wx,b,name='y')

y_=tf.subtract(x,b,name='y_')

with tf.Session() as sess:
    print('Intermediate result: Wx =', sess.run(Wx, feed_dict={x:[3,33]}))
    
    print('Final result Wx+b', sess.run(y, feed_dict={x:[5,50], b:[7,9]}))
    
    print('Intermediate specified : Wx+b =',sess.run(fetches=y, feed_dict={x:[100,1000],b:[7,9]}))
    
    print('Two results [Wx + b], x-b ', sess.run(fetches=[y,y_],feed_dict={x:[5,50], b:[7,9]}) )
    
writer = tf.summary.FileWriter('./tbout/another',sess.graph)
writer.close()

Intermediate result: Wx = [  30 3300]
Final result Wx+b [  57 5009]
Intermediate specified : Wx+b = [  1007 100009]
Two results [Wx + b], x-b  [array([  57, 5009], dtype=int32), array([-2, 41], dtype=int32)]


# Variables

There is usually a need to use variables that can be updated, `tf.Variable` provide for this.

Variables can only be used after initialization , 

1. Get initialization function
specifying variable names - `init=tf.variables_initializer([number,multiplier])` 
global - `init=tf.global_variables_initializer()`

2. Running the initialization function in a session
    `sess.run(init)`
    


In [20]:
import tensorflow as tf

number = tf.Variable(2)
multiplier=tf.Variable(3)
init=tf.variables_initializer([number,multiplier])
result=tf.multiply(number,multiplier,name='result')

with tf.Session() as sess:
        sess.run(init)
        for i in range(10):
            print("Result number * mutiplier = {}".format(sess.run(result)))
            print("Increment multiplier, new value={}".format(sess.run(multiplier.assign(multiplier+1))))
            
            

Result number * mutiplier = 6
Increment multiplier, new value=4
Result number * mutiplier = 8
Increment multiplier, new value=5
Result number * mutiplier = 10
Increment multiplier, new value=6
Result number * mutiplier = 12
Increment multiplier, new value=7
Result number * mutiplier = 14
Increment multiplier, new value=8
Result number * mutiplier = 16
Increment multiplier, new value=9
Result number * mutiplier = 18
Increment multiplier, new value=10
Result number * mutiplier = 20
Increment multiplier, new value=11
Result number * mutiplier = 22
Increment multiplier, new value=12
Result number * mutiplier = 24
Increment multiplier, new value=13


# Named Scope

Instead of using cluttered graph like below:
```
    Ax2_1=tf.multiply(A,tf.pow(x,2),name='Ax2_1')
    Bx=tf.mutiply(B,x, name='Bx')
    y1=tf.add_n([Ax2_1,Bx],name='y1')

    Ax2_2=tf.multiply(A,tf.pow(x,2),name='Ax2_2')
    Bx=tf.mutiply(B,x, name='Bx')
    y2=tf.add_n([Ax2_2,Bx],name='y2')

    y=y1+y2
```

group operations using named scope like the code below(notice it's possible to reuse the name Ax2 in different named scope) 

In [29]:
import tensorflow as tf

A=tf.constant([4],tf.int32, name='A')
B=tf.constant([4],tf.int32, name='B')
C=tf.constant([4],tf.int32, name='C')

x=tf.placeholder(tf.int32, name='x')

with tf.name_scope('Equation_1'):
    Ax2=tf.multiply(A,tf.pow(x,2),name='Ax2')
    Bx=tf.multiply(B,x, name='Bx')
    y1=tf.add_n([Ax2,Bx],name='y1')
    
with tf.name_scope('Equation_2'):
    Ax2=tf.multiply(A,tf.pow(x,2),name='Ax2')
    Bx=tf.multiply(B,x, name='Bx')
    y2=tf.add_n([Ax2,Bx],name='y2')
      
with tf.name_scope('Final_sum'):
    y=y1+y2
    
with tf.Session() as sess:
    print(sess.run(y, feed_dict={x:[10]}))
    
writer = tf.summary.FileWriter('./tbout/unclutterGrapWithNamedScope',sess.graph)
writer.close()

[880]


# Graph 

It's possible to have multiple graphs. e.g.

```
    g1=tf.Graph()
    
    with g1.as_default():
        with tf.Session() as sess:
```

# Interactive Session

Instead of using a session variable, we can use an interactive session especially within the python repl. For example :


In [31]:
import tensorflow as tf
sess=tf.InteractiveSession()

A = tf.placeholder(tf.int32, name='A')

y = A**2

y.eval(feed_dict={A:6})

36

# Application - Simple machine learning, linear regression

Basically minimize loss using a gradient descent optimizer

In [15]:
import tensorflow as tf

W=tf.Variable([.3],name='W',dtype=tf.float32)
b=tf.Variable([-.3],name='b',dtype=tf.float32)

x=tf.placeholder(tf.float32, name='x')
y=tf.placeholder(tf.float32, name='y')

model=W*x+b
loss=tf.reduce_sum(tf.square(model-y))
optimizer=tf.train.GradientDescentOptimizer(0.01)
fit=optimizer.minimize(loss)

x_train=[1,2,3,4]
y_train=[0,-1,-2,-3]
init= tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)    
    for i in range(1000):
        sess.run(fit,feed_dict={x:x_train,y:y_train})
    
    curr_W,curr_b,loss=sess.run([W,b,loss],feed_dict={x:x_train,y:y_train})
    print("W:{}, b:{}, loss:{}".format( curr_W,curr_b,loss))    
writer = tf.summary.FileWriter('./tbout/linearRegression',sess.graph)
writer.close()

W:[-0.9999969], b:[ 0.99999082], loss:5.699973826267524e-11
